In [18]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

## Maestría en Inteligencia Artificial Aplicada (MNA)
### Proyecto Integrador
### Dra. Grettel Barceló Alonso / Dr. Carlos Alberto Villaseñor Padilla
### Avance 5. Implementación de la Base de Vectores Final

### Integrantes
- A01794457 - Iossif Moises Palli Laura
- A01793984 - Brenda Zurazy Rodríguez Pérez
- A01794630 - Jesús Ramseths Echeverría Rivera

### 1. Carga y división de documentos

In [3]:
name_files = ['Ley de Fondos de Inversión', 'Ley para Regular las Instituciones de Tecnología Financiera',
              'Ley para Regular las Agrupaciones Financieras'] # Nombre de los archivos

In [4]:
# Aplicación de Separador Recursivo delimitado a 500 fragmentos
docs = []
for file in name_files:
    # Cargar un documento PDF y extraer sus páginas
    loader = PyPDFLoader(f'../base/{file}.pdf', extract_images=False)
    pages = loader.load()
    # Dividir el texto en fragmentos más pequeños
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    tmp_docs = text_splitter.split_documents(pages)
    docs.extend(tmp_docs)

In [5]:
print(f'Cantidad de Documentos: {len(docs)}')

Cantidad de Documentos: 3081


### 2. Preprocesamiento

In [6]:
# Limpiar el texto de los documentos
for doc in docs:
    # Se elimina espacios y saltos de línea al inicio y al final
    text = doc.page_content.strip()
    text = text.replace('\n', ' ')

    # Se wliminan espacios duplicados
    text = re.sub(r'[ \t]+', ' ', text)

    # Actualizar
    doc.page_content = text

# Filtrar documentos vacíos
docs = [doc for doc in docs if doc.page_content.strip() != '']

### 3. Creación de Base Vectorial

In [8]:
# Se usa este modelo por ser ligero en cuestión de recursos computacionales
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

In [10]:
# Definir la ruta donde se guardarán los embeddings
persist_path = "../database/embeddings_db"
# Especificación de la VectorDB
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_path=persist_path,
    serializer="parquet"
)
# Se crea la base
vector_store.persist()

### 4. Prueba de la Base Vectorial

In [13]:
# Se manda un prompt y se hace la búsqueda por similitud del coseno
prompt = "Información relacionada a los fondos de inversión en la CNBV"
documents = vector_store.similarity_search(prompt)

In [17]:
print('Resultado: \n')
print(documents[0].page_content)

Resultado: 

(3) Las sociedades operadoras de fondos de inversión, así como las distribuidoras y valuadoras de acciones de fondos de inversión que se ubiquen en lo dispuesto en esta fracción, deberán entregar a la Comisión, la información y documentación que acredite satis facer lo antes señalado, dentro de los quince días hábiles siguientes a que se encuentren en dicho supuesto. La Comisión tendrá un plazo de noventa días hábiles, contado a partir de que reciba la información y documentación
